In [88]:
import os
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data


os.environ["CUDA_VISIBLE_DEVICES"] = '0'   #指定第一块GPU可用
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5  # 程序最多只能占用指定gpu50%的显存
config.gpu_options.allow_growth = True      #程序按需申请内存
sess = tf.Session(config = config)


#载入自带数据集
mnist = input_data.read_data_sets("./MNIST_data/",one_hot=True)

 
test_data = mnist.train.images[:10].reshape((-1, 784))
test_label = mnist.train.labels[:10]


Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


In [89]:
tf.reset_default_graph()

# 输入图片是28*28
n_inputs = 28 #输入一行，一行有28个数据
max_time = 28 #一共28行
lstm_size = 100 #隐层单元
n_classes = 10 # 10个分类
batch_size = 50 #每批次50个样本
n_batch = mnist.train.num_examples // batch_size #计算一共有多少个批次
#这里的none表示第一个维度可以是任意的长度
## 命名为inputxx
x = tf.placeholder(tf.float32,[None,784], name='inputxx')
#正确的标签
y = tf.placeholder(tf.float32,[None,10])
#初始化权值
weights = tf.Variable(tf.truncated_normal([lstm_size, n_classes], stddev=0.1))
#初始化偏置值
biases = tf.Variable(tf.constant(0.1, shape=[n_classes]))
#定义RNN网络
def RNN(X,weights,biases,name):
    # inputs=[batch_size, max_time, n_inputs]
    inputs = tf.reshape(X,[-1,max_time,n_inputs])
    #定义LSTM基本CELL
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    # final_state[0]是cell state
    # final_state[1]是hidden_state
    outputs,final_state = tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32)
    results = tf.nn.softmax(tf.matmul(final_state[1],weights) + biases, name=name)
    return results
#计算RNN的返回结果  分别命名为output_pre和outputxx
prediction= RNN(x, weights, biases, name="output_pre") 
prediction_labels = tf.argmax(prediction, axis=1, name="outputxx")

#损失函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
#使用AdamOptimizer进行优化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#结果存放在一个布尔型列表中
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中最大的值所在的位置
#求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))#把correct_prediction变为float32类型



In [90]:
#初始化
res = 0
res_pre = 0
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(2):
        for batch in range(n_batch):
            batch_xs,batch_ys =  mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print ("Iter " + str(epoch) + ", Testing Accuracy= " + str(acc))
    preddd = sess.run(prediction, feed_dict={x: test_data, y: test_label})
    
    res_pre = preddd
    print(res_pre)
#     array_pre = sess.run(res_pre)
#     # 打印其数据类型与其值
#     print(type(array_pre))
#     print(array_pre)
    
    prediction_l = tf.argmax(preddd, axis=1)
    res = prediction_l
    array = sess.run(res)
    # 打印其数据类型与其值
    print(type(array))
    print(array)

    graph = tf.graph_util.convert_variables_to_constants(sess, sess.graph_def, ["inputxx","outputxx","output_pre"])
    tf.train.write_graph(graph, '.', 'offlineLSTM.pb', as_text=False)
print("__________ok_________")    

Iter 0, Testing Accuracy= 0.7436
Iter 1, Testing Accuracy= 0.863
[[  1.24786005e-04   2.64288887e-04   1.99980587e-02   9.71099854e-01
    2.49520817e-05   4.18117299e-04   4.52300628e-05   2.42086407e-03
    5.13480045e-04   5.09040989e-03]
 [  1.01817459e-04   3.83407285e-04   3.83723550e-03   9.88258600e-01
    2.57151914e-05   2.13995716e-03   2.01782732e-05   9.47472232e-04
    9.92877060e-04   3.29281297e-03]
 [  3.48399131e-04   2.04604992e-04   4.80323331e-04   3.85689083e-04
    7.53585398e-01   1.00939283e-02   1.59015658e-03   2.57629585e-02
    8.83968198e-04   2.06664503e-01]
 [  7.20905489e-04   3.76690528e-04   2.52732309e-04   1.07885980e-05
    4.16903931e-04   1.82727899e-03   9.95966077e-01   7.53562836e-06
    1.71552805e-04   2.49562872e-04]
 [  1.56430266e-04   9.97897625e-01   3.13562050e-04   2.29505808e-04
    6.19993716e-06   1.34421964e-04   6.28091395e-04   2.14157262e-04
    3.32128518e-04   8.78727005e-05]
 [  1.58869650e-03   6.16473480e-05   7.05729530e-

## 提取feature和label在Java下构建样本并对比验证结果正确性

In [91]:
print(res)
print(res.shape)
session = tf.Session()
# 张量转化为ndarray
array = session.run(res)
# 打印其数据类型与其值
print(type(array))
print(array)

Tensor("ArgMax_2:0", shape=(10,), dtype=int64)
(10,)
<class 'numpy.ndarray'>
[3 3 4 6 1 8 1 0 9 8]


In [92]:
print(prediction_labels.name)

outputxx:0


In [93]:
print(test_data[3])

[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.

In [94]:
print(test_label[3])

[ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]


In [82]:
print(test_data[1])

[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.

In [81]:
print(test_label[1])

[ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
